In [1]:
import sys
sys.path.append('/home/macul/insightface/src/symbols')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/')

In [2]:
import fresnet
import cv2
from collections import namedtuple

import mxnet as mx
from mxconv.mxresnet import MxResNet
from mxiter.mxiter import ImageMultiLabelIter,ImageMultiLabelIter_IndividualLabel
from mxloss.mxloss import MxLosses
import numpy as np
import argparse
import logging
import json
import os

def get_image(filename):
    img = cv2.imread(filename)  # read image in b,g,r order
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # change to r,g,b order
    img = cv2.resize(img, (112, 112))  # resize to 224*224 to fit model
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)  # change to (channel, height, width)
    img = img[np.newaxis, :]  # extend to (example, channel, heigth, width)
    return img

Batch = namedtuple('Batch', ['data'])

In [3]:
#model = MxResNet.build(1000, (3, 4, 14, 3), (64, 256, 512, 1024, 2048))
model = MxResNet.build(512, (3, 4, 6, 3), (64, 64, 128, 256, 512))
softmax_label = mx.symbol.Variable('softmax_label')
                                  

In [4]:
trainIter = mx.io.ImageRecordIter(
									path_imgrec='/media/macul/black/face_database_raw_data/ms1m_crop_112x112/rec/train.rec',
									data_shape=(3,112,112),
									batch_size=128,
                                    scale = 0.0078125,
                                    label_width=11)

In [5]:
trainIter = ImageMultiLabelIter_IndividualLabel(trainIter, label_idx=0, label_len=[1, 10], label_name=['softmax_label', 'landmark_gt'])

In [6]:
all_layers = model.get_internals()
all_layers.list_outputs()

['data',
 'stem_conv1_weight',
 'stem_conv1_output',
 'stem_bn2_gamma',
 'stem_bn2_beta',
 'stem_bn2_moving_mean',
 'stem_bn2_moving_var',
 'stem_bn2_output',
 'stem_relu1_gamma',
 'stem_relu1_output',
 'stage1_unit1_bn1_gamma',
 'stage1_unit1_bn1_beta',
 'stage1_unit1_bn1_moving_mean',
 'stage1_unit1_bn1_moving_var',
 'stage1_unit1_bn1_output',
 'stage1_unit1_prelu1_gamma',
 'stage1_unit1_prelu1_output',
 'stage1_unit1_conv1_weight',
 'stage1_unit1_conv1_output',
 'stage1_unit1_bn2_gamma',
 'stage1_unit1_bn2_beta',
 'stage1_unit1_bn2_moving_mean',
 'stage1_unit1_bn2_moving_var',
 'stage1_unit1_bn2_output',
 'stage1_unit1_prelu2_gamma',
 'stage1_unit1_prelu2_output',
 'stage1_unit1_conv2_weight',
 'stage1_unit1_conv2_output',
 'stage1_unit1_convr_weight',
 'stage1_unit1_convr_output',
 '_plus0_output',
 'stage1_unit2_bn1_gamma',
 'stage1_unit2_bn1_beta',
 'stage1_unit2_bn1_moving_mean',
 'stage1_unit2_bn1_moving_var',
 'stage1_unit2_bn1_output',
 'stage1_unit2_prelu1_gamma',
 'stage1_u

In [7]:
sym3 = all_layers['out_fc1_output']
mod3 = mx.mod.Module(symbol=sym3, context=mx.cpu())
mod3.bind(for_training=False, data_shapes=[('data', (1,3,112,112))])
mod3.init_params()

/home/macul/mx_venv/local/lib/python2.7/site-packages/mxnet/module/base_module.py:55: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
  warnings.warn(msg)
/home/macul/mx_venv/local/lib/python2.7/site-packages/mxnet/module/base_module.py:67: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [8]:
img = get_image('/home/macul/2018-10-15.png')

#output = mod3(Batch([mx.nd.array(img)]))
#batch = trainIter.next()
#mod3.forward(batch)
print mod3.get_outputs()[0].asnumpy()
mod3.predict(trainIter)

[[3.5882600e-24 4.5898130e-41 3.5882600e-24 4.5898130e-41 4.4851057e-30
  0.0000000e+00 4.4851057e-30 0.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000

TypeError: zip argument #1 must support iteration

In [10]:
mod3.get_outputs()[0].asnumpy().shape

(1, 512)

In [21]:
out = mod3.get_outputs()[0].asnumpy()
print(out.shape)

(1, 25088)


In [9]:
mx.visualization.print_summary(sym3, shape={'data':(1,3,112,112)})

________________________________________________________________________________________________________________________
Layer (type)                                        Output Shape            Param #     Previous Layer                  
data(null)                                          3x112x112               0                                           
________________________________________________________________________________________________________________________
stem_conv1(Convolution)                             64x112x112              1728        data                            
________________________________________________________________________________________________________________________
stem_bn2(BatchNorm)                                 64x112x112              128         stem_conv1                      
________________________________________________________________________________________________________________________
stem_relu1(LeakyReLU)           

In [20]:
512*49*512

12845056

In [40]:
sym = fresnet.get_symbol(1000, 50, 
        version_se=0, version_input=1, 
        version_output='E', version_unit=3,
        version_act='prelu')

0 1 E 3 prelu


In [41]:
all_layers = sym.get_internals()

In [42]:
all_layers.list_outputs()

['data',
 'id_output',
 '_minusscalar2_output',
 '_mulscalar2_output',
 'conv0_weight',
 'conv0_output',
 'bn0_gamma',
 'bn0_beta',
 'bn0_moving_mean',
 'bn0_moving_var',
 'bn0_output',
 'relu0_gamma',
 'relu0_output',
 'stage1_unit1_bn1_gamma',
 'stage1_unit1_bn1_beta',
 'stage1_unit1_bn1_moving_mean',
 'stage1_unit1_bn1_moving_var',
 'stage1_unit1_bn1_output',
 'stage1_unit1_conv1_weight',
 'stage1_unit1_conv1_output',
 'stage1_unit1_bn2_gamma',
 'stage1_unit1_bn2_beta',
 'stage1_unit1_bn2_moving_mean',
 'stage1_unit1_bn2_moving_var',
 'stage1_unit1_bn2_output',
 'stage1_unit1_relu1_gamma',
 'stage1_unit1_relu1_output',
 'stage1_unit1_conv2_weight',
 'stage1_unit1_conv2_output',
 'stage1_unit1_bn3_gamma',
 'stage1_unit1_bn3_beta',
 'stage1_unit1_bn3_moving_mean',
 'stage1_unit1_bn3_moving_var',
 'stage1_unit1_bn3_output',
 'stage1_unit1_conv1sc_weight',
 'stage1_unit1_conv1sc_output',
 'stage1_unit1_sc_gamma',
 'stage1_unit1_sc_beta',
 'stage1_unit1_sc_moving_mean',
 'stage1_unit1_sc

In [55]:
sym3 = all_layers['stage4_unit3_bn3_output']
mod3 = mx.mod.Module(symbol=sym3, label_names=None, context=mx.cpu())
mod3.bind(for_training=False, data_shapes=[('data', (1,3,448,448))])
mod3.init_params()
#mod3.set_params(arg_params, aux_params)

In [56]:

img = get_image('/home/macul/2018-10-15.png')

#output = mod3(Batch([mx.nd.array(img)]))
mod3.forward(Batch([mx.nd.array(img)]))

In [57]:

out = mod3.get_outputs()[0].asnumpy()
print(out.shape)

(1, 512, 7, 7)


In [17]:
512*7*7

25088

In [ ]:
all_label = mx.symbol.Variable('softmax_label')
gt_label = all_label
extra_loss = None
_weight = mx.symbol.Variable("fc7_weight", shape=(args.num_classes, args.emb_size), lr_mult=args.fc7_lr_mult, wd_mult=args.fc7_wd_mult)

s = args.margin_s #64
m = args.margin_m #0.5
assert s>0.0
assert m>=0.0
assert m<(math.pi/2)
_weight = mx.symbol.L2Normalization(_weight, mode='instance')
nembedding = mx.symbol.L2Normalization(embedding, mode='instance', name='fc1n')*s
fc7 = mx.sym.FullyConnected(data=nembedding, weight = _weight, no_bias = True, num_hidden=args.num_classes, name='fc7')
zy = mx.sym.pick(fc7, gt_label, axis=1)
cos_t = zy/s
cos_m = math.cos(m)
sin_m = math.sin(m)
mm = math.sin(math.pi-m)*m
#threshold = 0.0
threshold = math.cos(math.pi-m)
if args.easy_margin: #0
  cond = mx.symbol.Activation(data=cos_t, act_type='relu')
else:
  cond_v = cos_t - threshold
  cond = mx.symbol.Activation(data=cond_v, act_type='relu')
body = cos_t*cos_t
body = 1.0-body
sin_t = mx.sym.sqrt(body)
new_zy = cos_t*cos_m
b = sin_t*sin_m
new_zy = new_zy - b
new_zy = new_zy*s
if args.easy_margin:
  zy_keep = zy
else:
  zy_keep = zy - s*mm
new_zy = mx.sym.where(cond, new_zy, zy_keep)

diff = new_zy - zy
diff = mx.sym.expand_dims(diff, 1)
gt_one_hot = mx.sym.one_hot(gt_label, depth = args.num_classes, on_value = 1.0, off_value = 0.0)
body = mx.sym.broadcast_mul(gt_one_hot, diff)
fc7 = fc7+body    


out_list = [mx.symbol.BlockGrad(embedding)]
softmax = mx.symbol.SoftmaxOutput(data=fc7, label = gt_label, name='softmax', normalization='valid')
out_list.append(softmax)
if args.loss_type==6:
out_list.append(intra_loss)
if args.loss_type==7:
out_list.append(inter_loss)
#out_list.append(mx.sym.BlockGrad(counter_weight))
#out_list.append(intra_loss)
if args.ce_loss:
#ce_loss = mx.symbol.softmax_cross_entropy(data=fc7, label = gt_label, name='ce_loss')/args.per_batch_size
body = mx.symbol.SoftmaxActivation(data=fc7)
body = mx.symbol.log(body)
_label = mx.sym.one_hot(gt_label, depth = args.num_classes, on_value = -1.0, off_value = 0.0)
body = body*_label
ce_loss = mx.symbol.sum(body)/args.per_batch_size
out_list.append(mx.symbol.BlockGrad(ce_loss))
out = mx.symbol.Group(out_list)

In [4]:
model, argParams, auxParams = mx.model.load_checkpoint('/home/macul/libraries/mk_utils/mx_facerecog_resnet50/output/train_5/train_5', 1)

In [9]:
model

<Symbol group [softmax, lro, embedding]>

In [8]:
argParams.keys()

['stage4_unit2_prelu2_gamma',
 'stage3_unit2_bn3_beta',
 'stage2_unit2_bn1_beta',
 'stage3_unit5_bn3_gamma',
 'out_fc1_bias',
 'stage4_unit1_prelu3_gamma',
 'stage3_unit6_bn3_gamma',
 'stage3_unit3_conv1_weight',
 'stage2_unit1_bn3_gamma',
 'stage4_unit1_prelu1_gamma',
 'out_embedding_beta',
 'stage2_unit3_bn1_beta',
 'stage3_unit4_conv1_weight',
 'stage2_unit2_bn3_beta',
 'stage2_unit1_prelu2_gamma',
 'stage1_unit3_bn1_beta',
 'stage2_unit2_conv1_weight',
 'stage1_unit2_bn2_beta',
 'stage3_unit2_conv2_weight',
 'stage1_unit2_conv1_weight',
 'stem_bn1_gamma',
 'stage2_unit1_conv3_weight',
 'stage3_unit2_bn2_gamma',
 'stage1_unit1_conv3_weight',
 'stage1_unit1_conv1_weight',
 'out_embedding_gamma',
 'stage4_unit1_bn3_beta',
 'stage4_unit3_conv3_weight',
 'stage1_unit1_convr_weight',
 'regr_fc_weight',
 'stage2_unit2_conv2_weight',
 'stage1_unit3_bn3_beta',
 'stage2_unit4_prelu1_gamma',
 'stage3_unit6_prelu1_gamma',
 'stage1_unit3_bn2_gamma',
 'stage1_unit3_bn3_gamma',
 'stage2_unit1_pre